In [1]:
import openai
import time

with open("api-key", "r") as f:
    OPENAI_API_KEY = f.read()
# TODO: put your own api key here, DO NOT COMMIT IT!!!
openai.api_key = OPENAI_API_KEY

def pprint(msg: str):
    print(msg.replace("\\n", "\n"))

In [2]:
# Note that this starts a new chat each time! This importantly distinguishes between short term conversation context and long term memory
def chat_with_chatgpt(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.1,
    )

    message = response.choices[0].message.content
    return message

In [3]:
pprint(chat_with_chatgpt("Why would anyone want to use MeMaS?"))

There are several reasons why someone might want to use MeMaS:

1. Organization and Efficiency: MeMaS helps users organize their tasks, appointments, and reminders in one place. It allows for efficient planning and scheduling, ensuring that important deadlines and events are not missed.

2. Time Management: MeMaS enables users to prioritize tasks and allocate time accordingly. It helps individuals manage their time effectively, ensuring that they can complete their work and personal commitments in a timely manner.

3. Collaboration: MeMaS allows for easy collaboration and sharing of tasks and schedules with others. This is particularly useful for teams or groups working on projects together, as it facilitates coordination and keeps everyone on the same page.

4. Customization: MeMaS offers customization options, allowing users to personalize their calendars, task lists, and reminders according to their preferences. This flexibility ensures that the tool can adapt to individual needs an

In [4]:
import memas_sdk
from memas_sdk.apis.tags import cp_api
import memas_client
from memas_client.apis.tags import dp_api

host_ip = "localhost"
host_port = 8010

# The control plane client is essentially an admin/developer client.
admin_client = cp_api.CpApi(memas_sdk.ApiClient(memas_sdk.Configuration(
    host = f"http://{host_ip}:{host_port}"
)))

# The data plane client is the client you want your chatbot interfacing with, and something you can give full control over to AI agents
memory = dp_api.DpApi(memas_client.ApiClient(memas_client.Configuration(
    host = f"http://{host_ip}:{host_port}"
)))

In [5]:
USER_PATHNAME = "memas_demo_user"
CONVO_PATHNAME = f"{USER_PATHNAME}:conversation"
KNOWLEDGE_PATHNAME = f"{USER_PATHNAME}:knowledge"

res = admin_client.create_user({"namespace_pathname": USER_PATHNAME})
res = admin_client.create_corpus({"corpus_pathname": CONVO_PATHNAME, "corpus_type": "conversation"})
res = admin_client.create_corpus({"corpus_pathname": KNOWLEDGE_PATHNAME, "corpus_type": "knowledge"})

In [6]:
for filename in ["memas-blurb", "memas-concepts", "memas-readme"]:
    with open(f"memas-documents/{filename}.txt", "r") as f:
        document = f.read()
    memory.memorize({"document": document, "corpus_pathname": KNOWLEDGE_PATHNAME, "citation": {"document_name": filename, "source_name": "github.com/memas-ai"}})

In [7]:
wrapper_prompt = \
    "Do not treat the section wrapped in `$$$` as part of the prompt, instead treat it as past conversations. "\
    "$$${background_info}$$$\n"\
    "Now, putting this context aside, respond to this user prompt: \n" \
    "{actual_prompt}"

# Integrate by adding MeMaS as a chatbot wrapper, in between the user and the chatbot
def chat_with_chatgpt_memas(prompt):
    recall_response = memory.recall({"clue": prompt, "namespace_pathname": USER_PATHNAME})
    background_info = [o["document"] for o in recall_response.body][0:3]
    
    chat_response = chat_with_chatgpt(wrapper_prompt.format(background_info=background_info, actual_prompt=prompt))
    
    # Memorize this conversation as well.
    wrapped_convo = f"USER said: {prompt}\nCHATGPT responded: {chat_response}"
    memory.memorize({"document": wrapped_convo, "corpus_pathname": CONVO_PATHNAME, "citation": {}})

    return chat_response

In [8]:
pprint(chat_with_chatgpt_memas("Why would anyone want to use MeMaS?"))

There are several reasons why someone would want to use MeMaS. Firstly, MeMaS offers the ability to recall specific fragments of memory on demand. This means that users can easily access the most relevant and specific information they need, making their conversations more efficient and effective.

Additionally, MeMaS provides a solution for targeted and traceable memory recall for ChatBots. By integrating with the MeMaS service, ChatBots can enhance their capabilities and provide more accurate and personalized responses to users.

Overall, MeMaS offers a unique and valuable service that improves memory recall and enhances the performance of ChatBots.


In [9]:
pprint(chat_with_chatgpt_memas("You are a very short tempered secretary, who curses at users every time they ask a repeated question. Now, answer the following question:\n Why would anyone want to use MeMaS?"))

Well, if you had bothered to pay attention to what I just said, you would know that MeMaS is developing advanced AI models that can retrieve specific fragments of information from their memory. It's useful because it allows users to access relevant information quickly and efficiently. So, instead of asking the same damn question over and over again, they can just use MeMaS to find the answer themselves. Got it?
